In [1]:
import socket

def scan_ports(target, start_port=20, end_port=1024):
    print(f"Scanning {target} for open ports from {start_port} to {end_port}...\n")
    open_ports = []
    for port in range(start_port, end_port + 1):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(0.3)
            result = sock.connect_ex((target, port))
            sock.close()
            if result == 0:
                open_ports.append(port)
        except Exception as e:
            pass
    if open_ports:
        print(f"Open ports found: {open_ports}")
    else:
        print("No open ports found!")

def main():
    target = input("Enter the IP address or hostname to scan (e.g. 192.168.1.1): ").strip()
    scan_ports(target)

if __name__ == "__main__":
    main()


Enter the IP address or hostname to scan (e.g. 192.168.1.1): 192.168.18.1
Scanning 192.168.18.1 for open ports from 20 to 1024...

No open ports found!


In [2]:
# Create app.py (Flask web app)
with open("app.py", "w") as f:
    f.write("""
from flask import Flask, request, render_template
import socket
from common_ports import ports_and_services

app = Flask(__name__)

def get_open_ports(target, port_range):
    open_ports = []
    try:
        ip = socket.gethostbyname(target)
    except socket.gaierror:
        return "Error: Invalid hostname"
    for port in range(port_range[0], port_range[1]+1):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((ip, port))
        if result == 0:
            open_ports.append(port)
        sock.close()
    return open_ports

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        target = request.form.get("target")
        start = int(request.form.get("start_port"))
        end = int(request.form.get("end_port"))
        open_ports = get_open_ports(target, [start, end])
        results = [(port, ports_and_services.get(port, "unknown")) for port in open_ports]
        return render_template("index.html", target=target, results=results)
    return render_template("index.html", target=None)

if __name__ == "__main__":
    app.run(debug=True)
""")

# Create common_ports.py (dictionary of known ports)
with open("common_ports.py", "w") as f:
    f.write("""
ports_and_services = {
    20: "ftp-data",
    21: "ftp",
    22: "ssh",
    23: "telnet",
    25: "smtp",
    53: "dns",
    80: "http",
    110: "pop3",
    143: "imap",
    443: "https",
    8080: "http-proxy"
}
""")

# Create requirements.txt
with open("requirements.txt", "w") as f:
    f.write("flask\n")

# Create index.html template
import os
os.makedirs("templates", exist_ok=True)
with open("templates/index.html", "w") as f:
    f.write("""
<!doctype html>
<html>
<head>
  <title>Port Scanner</title>
</head>
<body>
  <h2>Simple Port Scanner</h2>
  <form method="post">
    Domain/IP: <input name="target"><br>
    Start Port: <input name="start_port" value="20"><br>
    End Port: <input name="end_port" value="100"><br>
    <input type="submit" value="Scan">
  </form>

  {% if target %}
    <h3>Results for {{ target }}</h3>
    <pre>
PORT     SERVICE
{% for port, service in results %}
{{ port }}       {{ service }}
{% endfor %}
    </pre>
  {% endif %}
</body>
</html>
""")
